In [1]:
from __future__ import print_function, absolute_import

import os
import sys
import time
import copy

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import cameras
import data_utils
import linear_model
import procrustes
import viz
import glob
import cdflib
sys.argv = sys.argv[:1]

import cv2


import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import data_process as data_process

import json 
from model import LinearModel, OptunaModel, weight_init
import torch.nn as nn
import utils as utils


sys.path.append(os.path.join(os.path.dirname('__file__'), "progress"))

from progress.bar import Bar as Bar

# Load Human3.6M Skeleton

import torch.nn as nn
import torch.optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.autograd import Variable

import pickle

tf.app.flags.DEFINE_string("action","All", "The action to train on. 'All' means all the actions")

# Directories
tf.app.flags.DEFINE_string("cameras_path","../data/h36m/metadata.xml", "File with h36m metadata, including cameras")

FLAGS = tf.app.flags.FLAGS


# Initiate Function

In [2]:
SUBJECT_IDS = [1,5,6,7,8,9,11]
this_file = os.path.dirname(os.path.realpath('__file__'))

#Load metadata.xml camera
rcams = cameras.load_cameras(os.path.join(this_file, FLAGS.cameras_path), SUBJECT_IDS)

In [3]:



# Joints in H3.6M -- data has 32 joints, but only 17 that move; these are the indices.
H36M_NAMES = ['']*32
H36M_NAMES[0]  = 'Hip'
H36M_NAMES[1]  = 'RHip'
H36M_NAMES[2]  = 'RKnee'
H36M_NAMES[3]  = 'RFoot'
H36M_NAMES[6]  = 'LHip'
H36M_NAMES[7]  = 'LKnee'
H36M_NAMES[8]  = 'LFoot'
H36M_NAMES[12] = 'Spine'
H36M_NAMES[13] = 'Thorax'
H36M_NAMES[14] = 'Neck/Nose'
H36M_NAMES[15] = 'Head'
H36M_NAMES[17] = 'LShoulder'
H36M_NAMES[18] = 'LElbow'
H36M_NAMES[19] = 'LWrist'
H36M_NAMES[25] = 'RShoulder'
H36M_NAMES[26] = 'RElbow'
H36M_NAMES[27] = 'RWrist'

index_alphapose={
    # Use 17 skeleton point
    "Nose": 0,
    "RShoulder": 6,
    "RElbow": 8,
    "RWrist": 10,
    "LShoulder": 5,
    "LElbow": 7,
    "LWrist": 9,
    "RHip": 12,
    "RKnee": 14,
    "RAnkle": 16,
    "LHip": 11,
    "LKnee": 13,
    "LAnkle": 15,
    "REye": 2,
    "LEye": 1,
    "REar": 4,
    "LEar": 3
}

index_mapping={
# Alpha Pose to Human 3.6M
"Hip": [20, 0],
"RHip": [12,1],
"RKnee": [14,2],
"RFoot": [16,3],
"LHip": [11,6],
"LKnee": [13,7],
"LFoot": [15,8],
"Spine": [19,12],
"Thorax": [18,13],
# "Nose": [14,0],
"Head": [17,15],
"LShoulder": [5,17],
"LElbow": [7,18],
"LWrist": [9,19],
"RShoulder": [6,25],
"RElbow": [8,26],
"RWrist": [10,27]
}



index_mapping_nose={
# Alpha Pose to Human 3.6M
"Hip": [20, 0],
"RHip": [12,1],
"RKnee": [14,2],
"RFoot": [16,3],
"LHip": [11,6],
"LKnee": [13,7],
"LFoot": [15,8],
"Spine": [19,12],
"Thorax": [18,13],
"Nose": [0,14],
"Head": [17,15],
"LShoulder": [5,17],
"LElbow": [7,18],
"LWrist": [9,19],
"RShoulder": [6,25],
"RElbow": [8,26],
"RWrist": [10,27]
}


def data_converter(data):
    data=data['keypoints']
    keypoints=[]
    kp_score=[]
    for a in range (0,len(data)):
        score=[]
        if ((a+3)%3==0):
            keypoints.append(data[a])
            keypoints.append(data[a+1])
        elif((a+1)%3==0):
            score=data[a]
            kp_score.append(score)

    return keypoints


In [4]:


def count_head(alpha_pose):
    x = (alpha_pose[index_alphapose['LEar']*2]+alpha_pose[index_alphapose['REar']*2])/2
    y = (alpha_pose[index_alphapose['LEar']*2+1]+alpha_pose[index_alphapose['REar']*2+1])/2
    return x,y


def count_thorax(alpha_pose):
    x = (alpha_pose[index_alphapose['LShoulder']*2]+alpha_pose[index_alphapose['RShoulder']*2])/2
    y = (alpha_pose[index_alphapose['LShoulder']*2+1]+alpha_pose[index_alphapose['RShoulder']*2+1])/2
    return x,y


def count_spine(alpha_pose):
    hip_x,hip_y=count_hip(alpha_pose)
    thorax_x,thorax_y=count_thorax(alpha_pose)
    x = (hip_x+thorax_x)/2
    y = (hip_y+thorax_y)/2
    return x,y

def count_hip(alpha_pose):
    x = (alpha_pose[index_alphapose['LHip']*2]+alpha_pose[index_alphapose['RHip']*2])/2
    y = (alpha_pose[index_alphapose['LHip']*2+1]+alpha_pose[index_alphapose['RHip']*2+1])/2
    return x,y

def add_features(alpha_pose):
    #Count Head
    head_x,head_y=count_head(alpha_pose)
    alpha_pose=np.append(alpha_pose,(head_x,head_y))
    
    #Count Thorax
    thorax_x,thorax_y=count_thorax(alpha_pose)
    alpha_pose=np.append(alpha_pose,(thorax_x,thorax_y))
 
    
    #Count Spine
    spine_x,spine_y=count_spine(alpha_pose)
    alpha_pose=np.append(alpha_pose,(spine_x,spine_y))
    
    #Count Hip
    hip_x,hip_y=count_hip(alpha_pose)
    alpha_pose=np.append(alpha_pose,(hip_x,hip_y))
    
    return alpha_pose

In [5]:
#This function is not includding nose for 2D to 3D
def map_alpha_to_human(alpha_pose):
    alpha_pose=add_features(alpha_pose)
    temp_list = [None] * 64
    for a,b in index_mapping.items():
        temp_list[b[1]*2]=alpha_pose[b[0]*2]
        temp_list[b[1]*2+1]=alpha_pose[b[0]*2+1]
    human36m=np.asarray(temp_list)
    return human36m

#This function is includding nose for classification
def map_alpha_to_human_classification(alpha_pose):
    alpha_pose=add_features(alpha_pose)
    temp_list = [None] * 64
    for a,b in index_mapping_nose.items():
        temp_list[b[1]*2]=alpha_pose[b[0]*2]
        temp_list[b[1]*2+1]=alpha_pose[b[0]*2+1]
    human36m=np.asarray(temp_list)
    return human36m


def map_alpha_to_human_classification_json(path):
    # Opening JSON file 
    f = open(path) 
    converted=[]
    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 

    for dat in data:
        convert=np.asarray(data_converter(dat))
        human36m_alpha_example=map_alpha_to_human_classification(convert)
        human36m_alpha_example=human36m_alpha_example.astype('float')
        converted.append(human36m_alpha_example)

    converted=np.asarray(converted) 
        
    # Closing file 
    f.close() 
    
    return converted

# Do 3D Prediction from Custom Video

## Using Created Statistic Dictionary

In [6]:
normalize=True
actions = data_utils.define_actions( FLAGS.action )
# Human3.6m IDs for training and testing
TRAIN_SUBJECTS = [1,5,6,7,8]
TEST_SUBJECTS  = [9,11]


stat_3D = torch.load('../data/stat_3d.pth.tar')
stat_2D = torch.load('../data/stat_2d.pth.tar')




In [7]:

data_mean_2d, data_std_2d, dim_to_ignore_2d, dim_to_use_2d = stat_2D['mean'],stat_2D['std'],stat_2D['dim_ignore'],stat_2D['dim_use']
data_mean_3d, data_std_3d, dim_to_ignore_3d, dim_to_use_3d = stat_3D['mean'],stat_3D['std'],stat_3D['dim_ignore'],stat_3D['dim_use']

In [8]:
#All the json data
#load json data using json,load(f)
#!/usr/bin/env python
# -*- coding: utf-8 -*-


class Human36M_testing(Dataset):
    def __init__(self, skeleton,many=False):
        """
        :param actions: list of actions to use
        :param data_path: path to dataset
        :param use_hg: use stacked hourglass detections
        :param is_train: load train/test dataset
        """

        self.test_inp, self.test_out = [], []

        # loading data
        # load test data
       
        if many:
            num_f= skeleton.shape
            for i in range(num_f[0]):
                self.test_inp.append(skeleton[i])
        else:
            self.test_inp.append(skeleton)


    def __getitem__(self, index):
        inputs = torch.from_numpy(self.test_inp[index]).float()

        return inputs

    def __len__(self):
        return len(self.test_inp)
    
def normalize_single_data(data, data_mean, data_std, dim_to_use ):
    """Normalizes a dictionary of poses

    Args
    data: dictionary where values are
    data_mean: np vector with the mean of the data
    data_std: np vector with the standard deviation of the data
    dim_to_use: list of dimensions to keep in the data
    Returns
    data_out: dictionary with same keys as data, but values have been normalized
    """

    data= data[dim_to_use]
    mu = data_mean[dim_to_use]
    stddev = data_std[dim_to_use]
    data_out= np.divide( (data - mu), stddev )

    return data_out
def create_datatest(data):
    converted=[]
    for dat in data:
        convert=np.asarray(data_converter(dat))
        human36m_alpha_example=map_alpha_to_human(convert)
        normalized=normalize_single_data(human36m_alpha_example,data_mean_2d,data_std_2d,dim_to_use_2d)
        normalized=normalized.astype('float')
        converted.append(normalized)

    converted=np.asarray(converted) 
    test_loader = DataLoader(
        dataset=Human36M_testing(converted,True),
        batch_size=1024,
        shuffle=False,
        num_workers=0,
        pin_memory=True)
    return test_loader

# Load Model

In [9]:


model_path='../checkpoint/optuna/ckpt_best_1.pth.tar'
# create model
print(">>> creating model")
#model = LinearModel()
model = OptunaModel()
model = model.cuda()
model.apply(weight_init)
print(">>> total params: {:.2f}M".format(sum(p.numel() for p in model.parameters()) / 1000000.0))
criterion = nn.MSELoss(size_average=True).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

print(">>> loading ckpt from '{}'".format('../../3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'))
ckpt = torch.load(model_path)
start_epoch = ckpt['epoch']
err_best = ckpt['err']
glob_step = ckpt['step']
lr_now = ckpt['lr']
model.load_state_dict(ckpt['state_dict'])
optimizer.load_state_dict(ckpt['optimizer'])
print(">>> ckpt loaded (epoch: {} | err: {})".format(start_epoch, err_best))

new_stat_3d={}
new_stat_3d['mean']=data_mean_3d
new_stat_3d['std']=data_std_3d
new_stat_3d['dim_use']=dim_to_use_3d
new_stat_3d['dim_ignore']=dim_to_ignore_3d
    
def test(test_loader, model, criterion, stat_3d, procrustes=False):
    losses = utils.AverageMeter()
    model.eval()

    all_dist = []
    pred_result=[]
    start = time.time()
    batch_time = 0
    bar = Bar('>>>', fill='>', max=len(test_loader))

    for i, inps in enumerate(test_loader):
        inputs = Variable(inps.cuda())
        
        with torch.no_grad():
            outputs = model(inputs)

        # calculate erruracy
        print(outputs.shape)
        outputs_unnorm = data_process.unNormalizeData(outputs.data.cpu().numpy(), stat_3d['mean'], stat_3d['std'], stat_3d['dim_use'])

        # remove dim ignored
        dim_use = np.hstack((np.arange(3), stat_3d['dim_use']))

        outputs_use = outputs_unnorm[:, dim_use]
        pred_result.append(outputs_unnorm)
        
        # update summary
        if (i + 1) % 100 == 0:
            batch_time = time.time() - start
            start = time.time()

        bar.suffix = '({batch}/{size}) | batch: {batchtime:.4}ms | Total: {ttl} | ETA: {eta:} | loss: {loss:.6f}' \
            .format(batch=i + 1,
                    size=len(test_loader),
                    batchtime=batch_time * 10.0,
                    ttl=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg)
        bar.next()
    bar.finish()
    return pred_result

# Combine prediction from each batch into one prediction
def combine_prediction(pred_result_all):
    prediction_list = []
    for pred in pred_result_all:
        for pre in pred:
            prediction_list.append(pre)
    prediction_list=np.asarray(prediction_list)
    return prediction_list


def correct_3D(poses3d_input,poses2d_normalized):
    _max = 0
    _min = 10000
    poses3d=np.copy(poses3d_input)
    
    spine_x = poses2d_normalized[0][24]
    spine_y = poses2d_normalized[0][25]
            
    
    for i in range(poses3d.shape[0]):

        for j in range(32):

            tmp = poses3d[i][j * 3 + 2]
            poses3d[i][j * 3 + 2] = poses3d[i][j * 3 + 1]
            poses3d[i][j * 3 + 1] = tmp
            if poses3d[i][j * 3 + 2] > _max:
                _max = poses3d[i][j * 3 + 2]
                print("_max: ",_max)
            if poses3d[i][j * 3 + 2] < _min:
                _min = poses3d[i][j * 3 + 2]
                print("_min: ",_min)

    for i in range(poses3d.shape[0]):
        for j in range(32):
            poses3d[i][j * 3 + 2] = _max - poses3d[i][j * 3 + 2] + _min
            poses3d[i][j * 3] += (spine_x - 630)
            poses3d[i][j * 3 + 2] += (500 - spine_y)

    return poses3d

def inferencealphaposeto3D(path,fixing=False,save_npy=False):
    # Opening JSON file 
    f = open(path) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 


    # Closing file 
    f.close() 
    
    #Create Datatest for 2D to 3D Inference
    all_test_data=create_datatest(data)
    
    #Doing Inference
    pred_result_all=test(all_test_data, model, criterion, new_stat_3d) #All
    
    #Combine Prediction Result
    prediction_list=combine_prediction(pred_result_all)
    
    if fixing:
        #Fixing for unity
        test_2d_normalized = np.asarray(all_test_data.dataset.test_inp) 
        fixed=correct_3D(prediction_list,test_2d_normalized)
    else:
        fixed=prediction_list
    
    if save_npy:
        base=os.path.basename(path)
        base=os.path.splitext(base)[0]

        with open('../inference_result_npy/'+base+'.npy', 'wb') as f:
            np.save(f, fixed)
    
    return fixed



def save_to_json(result_3D, input_path, output_path):
    f = open(input_path) 
    dim_use = np.hstack((np.arange(3), dim_to_use_3d))

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 
    
    for a,b in zip(data,result_3D):
        a['keypoints']=b[dim_use].tolist()
        a['visualize']=b.tolist()
        
    with open(output_path, 'w') as fp:
        fp.write(json.dumps(data))
    
def save_to_json_2D(result_2D, input_path, output_path):
    f = open(input_path) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 
    
    for a,b in zip(data,result_2D):
        a['keypoints']=b[dim_to_use_2d].tolist()
        
    with open(output_path, 'w') as fp:
        fp.write(json.dumps(data))
        
def save_to_json_original2D(result_2D, input_path, output_path):
    f = open(input_path) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 
    
    a=dim_to_use_2d.tolist()
    a.insert(18,28)
    a.insert(19,29)
    dim_to_use_2d_nose=np.asarray(a)

    for a,b in zip(data,result_2D):
        a['keypoints']=b[dim_to_use_2d_nose].tolist()
        
    with open(output_path, 'w') as fp:
        fp.write(json.dumps(data))

>>> creating model
>>> total params: 10.60M
>>> loading ckpt from '../../3d_pose_baseline_pytorch/checkpoint/test/ckpt_best.pth.tar'
>>> ckpt loaded (epoch: 29 | err: 40.643307541837174)


D:\Project Lab\THESIS_FALL_DETECTION\2D to 3D Pose\3d_pose_baseline_pytorch\src\model.py:11: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)
C:\Users\hutom\anaconda3\envs\3DPose\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


Inference to json to three kinds:
1. Original 3D Baseline
2. 3D After doing proscrutes to another angle
3. 2D Mapping after doing proscrutes to another angle
4. Eucledian normalization of 2D Mapping after doing proscrutes

## Original 3D Baseline

In [18]:
%%time
path='../json_test/taoyuan_angle2.json'
base= os.path.splitext(os.path.basename(path))[0]
inference_result=inferencealphaposeto3D(path,fixing=False,save_npy=True)

torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([1024, 48])
torch.Size([967, 48])
Wall time: 766 ms


In [19]:
output_path='../inference_result_npy/'+base+'_3D_Original.json'
save_to_json(inference_result, path, output_path)

## Original 2D Human3.6m

This will save alpha pose in human3.6m format that have nose inside

In [20]:
output_path='../inference_result_npy/'+base+'_2D_Original.json'
mapping_result=map_alpha_to_human_classification_json(path)
save_to_json_original2D(mapping_result, path, output_path)

## 3D After doing proscrutes to another angle

In [21]:

def find_transformation_3D(skeleton_a,skeleton_b,fullskeleton):
    
    # remove dim ignored
    dim_use = np.hstack((np.arange(3), new_stat_3d['dim_use']))

    gt = skeleton_a[dim_use]
    out = skeleton_b[dim_use]
    gt = gt.reshape(-1, 3)
    out = out.reshape(-1, 3)
    _, Z, T, b, c = get_transformation(gt, out, True)
    transformation_value={"T":T,"b":b,"c":c}
    
    skeleton3D=[]
    for i,skeleton in enumerate(fullskeleton):
        skeleton = skeleton.reshape(-1, 3)
        skeleton = (b * skeleton.dot(T)) + c
        skeleton3D.append(skeleton)
    skeleton3D=np.asarray(skeleton3D)
    skeleton3D=skeleton3D.reshape(skeleton3D.shape[0],96)

    return  skeleton3D,transformation_value
    
def find_pair_transformation(path_1,pose_class_1,image_id_1,path_2,pose_class_2,image_id_2):
    with open(path_1, 'rb') as f:
        angle1 = json.load(f)
    with open(path_2, 'rb') as f:
        angle2 = json.load(f)
    
    angle1_temp=[]
    angle1_all=[]

    for a in angle1:
        if (a['pose_class']==pose_class_1) and (a['image_id']==image_id_1):
            angle1_temp.append(a)
    

    angle2_before=[]
    angle2_all=[]
    for a in angle2:
        if (a['pose_class']==pose_class_2) and (a['image_id']==image_id_2):
            angle2_before.append(a)
        angle2_all.append(a['visualize'])
    
    angle2_transformed_3D,transformation_value=find_transformation_3D(np.asarray(angle1_temp[0]['visualize']),
                                             np.asarray(angle2_before[0]['visualize']),
                                             np.asarray(angle2_all))
    

    return angle2_transformed_3D,transformation_value

def get_transformation(X, Y, compute_optimal_scale=True):
    muX = X.mean(0)
    muY = Y.mean(0)

    X0 = X - muX
    Y0 = Y - muY

    ssX = (X0 ** 2.).sum()
    ssY = (Y0 ** 2.).sum()

    # centred Frobenius norm
    normX = np.sqrt(ssX)
    normY = np.sqrt(ssY)

    # scale to equal (unit) norm
    X0 = X0 / normX
    Y0 = Y0 / normY

    # optimum rotation matrix of Y
    A = np.dot(X0.T, Y0)
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    V = Vt.T
    T = np.dot(V, U.T)

    # Make sure we have a rotation
    detT = np.linalg.det(T)
    V[:, -1] *= np.sign(detT)
    s[-1] *= np.sign(detT)
    T = np.dot(V, U.T)

    traceTA = s.sum()

    if compute_optimal_scale:  # Compute optimum scaling of Y.
        b = traceTA * normX / normY
        d = 1 - traceTA ** 2
        Z = normX * traceTA * np.dot(Y0, T) + muX
    else:  # If no scaling allowed
        b = 1
        d = 1 + ssY / ssX - 2 * traceTA * normY / normX
        Z = normY * np.dot(Y0, T) + muX

    c = muX - b * np.dot(muY, T)

    return d, Z, T, b, c

def save_transformation(transformed_value,output_path,path_1,pose_class_1,image_id_1,path_2,pose_class_2,image_id_2):
    base_1= os.path.splitext(os.path.basename(path_1))[0]
    base_2= os.path.splitext(os.path.basename(path_2))[0]
    transformed_value['Reference_Path']=base_1
    transformed_value['Reference_Pose_Class']=pose_class_1
    transformed_value['Reference_Image_ID']=image_id_1
    transformed_value['Transformed_Path']=base_2
    transformed_value['Transformed_Pose_Class']=pose_class_2
    transformed_value['Transformed_Image_ID']=image_id_2
    
    

    try:
        import cPickle as pickle
    except ImportError:  # Python 3.x
        import pickle
        
    with open(output_path, 'wb') as fp:
        pickle.dump(transformed_value, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
%%time
#Reference
path_1='../inference_result_npy/taoyuan_angle1_3D_Original.json'
pose_class_1="Standing_01"
image_id_1= '00631.png'

#Want to transform
path_2='../inference_result_npy/taoyuan_angle2_3D_Original.json'
pose_class_2="Standing_02"
image_id_2= '00646.png'

base_1= os.path.splitext(os.path.basename(path_1))[0]
base_2= os.path.splitext(os.path.basename(path_2))[0]


transformed,transformation_value=find_pair_transformation(path_1,pose_class_1,image_id_1,path_2,pose_class_2,image_id_2)

Wall time: 1.38 s


In [28]:
output_path='../inference_result_npy/'+base_2+" to "+base_1+'_proscrustes.json'
save_to_json(transformed, path, output_path)

In [29]:
output_path='../inference_result_npy/'+base_2+" to "+base_1+'_transformationvalue.pickle'
save_transformation(transformation_value, output_path,path_1,pose_class_1,image_id_1,path_2,pose_class_2,image_id_2)

## 2D Mapping after doing proscrutes to another angle

In [ ]:
import h5py

def load_cameras(bpath='cameras.h5', subjects=None):
    """
    :param bpath: *.h5
    :param subjects:
    :return: (dict)
    """

    if subjects is None:
        subjects = [1, 5, 6, 7, 8, 9, 11]
    rcams = {}

    with h5py.File(bpath, 'r') as hf:
        for s in subjects:
            for c in range(4):  # There are 4 cameras in human3.6m
                a = load_camera_params(hf, 'subject%d/camera%d/{0}' % (s, c + 1))
                rcams[(s, c + 1)] = a

    return rcams


def map3dto2dcamera( poses_set, cams, ncams=4 ):
    """
    Project 3d poses using camera parameters

    cams: dictionary with camera parameters
    ncams: number of cameras per subject

    """

    for cam in range( ncams ):
        R, T, f, c, k, p, name = cams[ (11, cam+1) ]
        pts2d, _, _, _, _ = project_point_radial( np.reshape(poses_set, [-1, 3]), R, T, f, c, k, p )

        pts2d = np.reshape( pts2d, [-1, len(H36M_NAMES)*2] )

    return pts2d

def project_point_radial(P, R, T, f, c, k, p):
    """
    Args
    P: Nx3 points in world coordinates
    R: 3x3 Camera rotation matrix
    T: 3x1 Camera translation parameters
    f: 2x1 (scalar) Camera focal length
    c: 2x1 Camera center
    k: 3x1 Camera radial distortion coefficients
    p: 2x1 Camera tangential distortion coefficients
    Returns
    Proj: Nx2 points in pixel space
    D: 1xN depth of each point in camera space
    radial: 1xN radial distortion per point
    tan: 1xN tangential distortion per point
    r2: 1xN squared radius of the projected points before distortion
    """

    # P is a matrix of 3-dimensional points
    assert len(P.shape) == 2
    assert P.shape[1] == 3

    N = P.shape[0]
    X = R.dot(P.T - T)  # rotate and translate
    XX = X[:2, :] / X[2, :]  # 2x16
    r2 = XX[0, :] ** 2 + XX[1, :] ** 2  # 16,

    radial = 1 + np.einsum('ij,ij->j', np.tile(k, (1, N)), np.array([r2, r2 ** 2, r2 ** 3]))  # 16,
    tan = p[0] * XX[1, :] + p[1] * XX[0, :]  # 16,

    tm = np.outer(np.array([p[1], p[0]]).reshape(-1), r2)  # 2x16

    XXX = XX * np.tile(radial + tan, (2, 1)) + tm  # 2x16

    Proj = (f * XXX) + c  # 2x16
    Proj = Proj.T

    D = X[2, ]

    return Proj, D, radial, tan, r2


def mapto2D(skeletons,rcams):
    """

    """
    skeleton2D=[]
    for i,skeleton in enumerate(skeletons):
        mapped=map3dto2dcamera(skeleton,rcams,4)
        skeleton2D.append(mapped)
    skeleton2D=np.asarray(skeleton2D)
    skeleton2D=skeleton2D.reshape(skeleton2D.shape[0],64)
    return skeleton2D

SUBJECT_IDS = [1,5,6,7,8,9,11]
this_file = os.path.dirname(os.path.realpath('__file__'))
rcams = cameras.load_cameras(os.path.join(this_file, "../data/h36m/metadata.xml"), SUBJECT_IDS)


In [ ]:
%%time
skeleton2d=mapto2D(transformed,rcams)


In [ ]:
output_path='../inference_result_npy/'+base+'_2D_Proscrutes.json'
save_to_json_2D(skeleton2d, path, output_path)

## Normalize using Eucledian Distances

In [ ]:
index_human36m={
    # Use 16 skeleton point
    "Pelvis": 0,
    "Head": 15,
    "Thorax":13,
    "RShoulder": 25,
    "RElbow": 26,
    "RWrist": 27,
    "LShoulder": 17,
    "LElbow": 18,
    "LWrist": 19,
    "RHip": 1,
    "RKnee": 2,
    "RAnkle": 3,
    "LHip": 6,
    "LKnee": 7,
    "LAnkle": 8,
    "Spine" : 12
}

def euclidean_dist(a, b):
    # This function calculates the euclidean distance between 2 point in 2-D coordinates
    # if one of two points is (0,0), dist = 0
    # a, b: input array with dimension: m, 2
    # m: number of samples
    # 2: x and y coordinate
    try:
        if (a.shape[1] == 2 and a.shape == b.shape):
            # check if element of a and b is (0,0)
            bol_a = (a[:,0] != 0).astype(int)
            bol_b = (b[:,0] != 0).astype(int)
            dist = np.linalg.norm(a-b, axis=1)
            return((dist*bol_a*bol_b).reshape(a.shape[0],1))
    except:
        print("[Error]: Check dimension of input vector")
        return 0


def norm_human36m(X):
    num_sample = X.shape[0]
    # Keypoints
    Pelvis = X[:,index_human36m['Pelvis']*2:index_human36m['Pelvis']*2+2]
    Head = X[:,index_human36m['Head']*2:index_human36m['Head']*2+2]
    Thorax = X[:,index_human36m['Thorax']*2:index_human36m['Thorax']*2+2]
    RShoulder = X[:,index_human36m['RShoulder']*2:index_human36m['RShoulder']*2+2]
    RElbow = X[:,index_human36m['RElbow']*2:index_human36m['RElbow']*2+2]
    RWrist = X[:,index_human36m['RWrist']*2:index_human36m['RWrist']*2+2]
    LShoulder = X[:,index_human36m['LShoulder']*2:index_human36m['LShoulder']*2+2]
    LElbow = X[:,index_human36m['LElbow']*2:index_human36m['LElbow']*2+2]
    LWrist = X[:,index_human36m['LWrist']*2:index_human36m['LWrist']*2+2]
    RHip = X[:,index_human36m['RHip']*2:index_human36m['RHip']*2+2]
    RKnee = X[:,index_human36m['RKnee']*2:index_human36m['RKnee']*2+2]
    RAnkle = X[:,index_human36m['RAnkle']*2:index_human36m['RAnkle']*2+2]
    LHip = X[:,index_human36m['LHip']*2:index_human36m['LHip']*2+2]
    LKnee = X[:,index_human36m['LKnee']*2:index_human36m['LKnee']*2+2]
    LAnkle = X[:,index_human36m['LAnkle']*2:index_human36m['LAnkle']*2+2]


    # Length of head
    length_Head_Thorax = euclidean_dist(Head, Thorax)
    length_head      = np.maximum.reduce([length_Head_Thorax])

    # Length of torso
    length_Thorax_LHip = euclidean_dist(Thorax, LHip)
    length_Thorax_RHip = euclidean_dist(Thorax, RHip)
    length_torso     = np.maximum(length_Thorax_LHip, length_Thorax_RHip)

    # Length of right leg
    length_leg_right = euclidean_dist(RHip, RKnee) + euclidean_dist(RKnee, RAnkle)
  
    # Length of left leg
    length_leg_left = euclidean_dist(LHip, LKnee) + euclidean_dist(LKnee, LAnkle)


    # Length of leg
    length_leg = np.maximum(length_leg_right, length_leg_left)

    # Length of body
    length_body = length_head + length_torso + length_leg
    
    # Check all samples have length_body of 0
    length_chk = (length_body > 0).astype(int)
    
    # Check keypoints at origin
    keypoints_chk = (X > 0).astype(int)
    
    chk = length_chk * keypoints_chk
    
    # Set all length_body of 0 to 1 (to avoid division by 0)
    length_body[length_body == 0] = 1
    
    # The center of gravity
    num_pts = (X[:, 0::2] > 0).sum(1).reshape(num_sample,1)

    centr_x = X[:, 0::2].sum(1).reshape(num_sample,1) / num_pts
    centr_y = X[:, 1::2].sum(1).reshape(num_sample,1) / num_pts

    # The  coordinates  are  normalized relative to the length of the body and the center of gravity
    X_norm_x = (X[:, 0::2] - centr_x) / length_body
    X_norm_y = (X[:, 1::2] - centr_y) / length_body
    
    # Stack 1st element x and y together
    X_norm = np.column_stack((X_norm_x[:,:1], X_norm_y[:,:1]))
        
    for i in range(1, X.shape[1]//2):
        X_norm = np.column_stack((X_norm, X_norm_x[:,i:i+1], X_norm_y[:,i:i+1]))
    
    # Set all samples have length_body of 0 to origin (0, 0)
    X_norm = X_norm * chk
    
    return X_norm

def euclidean_dist_3D(a, b):

    try:
        if (a.shape[1] == 3 and a.shape == b.shape):
            # check if element of a and b is (0,0)
            bol_a = (a[:,0] != 0).astype(int)
            bol_b = (b[:,0] != 0).astype(int)
            dist = np.linalg.norm(a-b, axis=1)
            return((dist*bol_a*bol_b).reshape(a.shape[0],1))
    except:
        print("[Error]: Check dimension of input vector")
        return 0


def norm_human36m_3D(X):
    num_sample = X.shape[0]
    # Keypoints
    Pelvis = X[:,index_human36m['Pelvis']*3:index_human36m['Pelvis']*3+3]
    Head = X[:,index_human36m['Head']*3:index_human36m['Head']*3+3]
    Thorax = X[:,index_human36m['Thorax']*3:index_human36m['Thorax']*3+3]
    RShoulder = X[:,index_human36m['RShoulder']*3:index_human36m['RShoulder']*3+3]
    RElbow = X[:,index_human36m['RElbow']*3:index_human36m['RElbow']*3+3]
    RWrist = X[:,index_human36m['RWrist']*3:index_human36m['RWrist']*3+3]
    LShoulder = X[:,index_human36m['LShoulder']*3:index_human36m['LShoulder']*3+3]
    LElbow = X[:,index_human36m['LElbow']*3:index_human36m['LElbow']*3+3]
    LWrist = X[:,index_human36m['LWrist']*3:index_human36m['LWrist']*3+3]
    RHip = X[:,index_human36m['RHip']*3:index_human36m['RHip']*3+3]
    RKnee = X[:,index_human36m['RKnee']*3:index_human36m['RKnee']*3+3]
    RAnkle = X[:,index_human36m['RAnkle']*3:index_human36m['RAnkle']*3+3]
    LHip = X[:,index_human36m['LHip']*3:index_human36m['LHip']*3+3]
    LKnee = X[:,index_human36m['LKnee']*3:index_human36m['LKnee']*3+3]
    LAnkle = X[:,index_human36m['LAnkle']*3:index_human36m['LAnkle']*3+3]


    # Length of head
    length_Head_Thorax = euclidean_dist_3D(Head, Thorax)
    length_head      = np.maximum.reduce([length_Head_Thorax])

    # Length of torso
    length_Thorax_LHip = euclidean_dist_3D(Thorax, LHip)
    length_Thorax_RHip = euclidean_dist_3D(Thorax, RHip)
    length_torso     = np.maximum(length_Thorax_LHip, length_Thorax_RHip)

    # Length of right leg
    length_leg_right = euclidean_dist_3D(RHip, RKnee) + euclidean_dist_3D(RKnee, RAnkle)
  
    # Length of left leg
    length_leg_left = euclidean_dist_3D(LHip, LKnee) + euclidean_dist_3D(LKnee, LAnkle)


    # Length of leg
    length_leg = np.maximum(length_leg_right, length_leg_left)

    # Length of body
    length_body = length_head + length_torso + length_leg
    
    # Check all samples have length_body of 0
    length_chk = (length_body > 0).astype(int)
    
    # Check keypoints at origin
    keypoints_chk = (X > 0).astype(int)
    
    chk = length_chk * keypoints_chk
    
    # Set all length_body of 0 to 1 (to avoid division by 0)
    length_body[length_body == 0] = 1
    
    # The center of gravity
    num_pts = np.full((num_sample, 1), 32)
    print(num_pts)
    centr_x = X[:, 0::3].sum(1).reshape(num_sample,1) / num_pts
    centr_y = X[:, 1::3].sum(1).reshape(num_sample,1) / num_pts
    centr_z = X[:, 2::3].sum(1).reshape(num_sample,1) / num_pts

    # The  coordinates  are  normalized relative to the length of the body and the center of gravity
    X_norm_x = (X[:, 0::3] - centr_x) / length_body
    X_norm_y = (X[:, 1::3] - centr_y) / length_body
    X_norm_z = (X[:, 2::3] - centr_z) / length_body

    # Stack 1st element x and y together
    X_norm = np.column_stack((X_norm_x[:,:1], X_norm_y[:,:1], X_norm_z[:,:1]))
        
    for i in range(1, X.shape[1]//3):
        X_norm = np.column_stack((X_norm, X_norm_x[:,i:i+1], X_norm_y[:,i:i+1], X_norm_z[:,i:i+1]))
    
    # Set all samples have length_body of 0 to origin (0, 0)
    X_norm = X_norm * chk
    
    return X_norm

In [ ]:
%%time
normalized_eucledianhuman36m=norm_human36m(skeleton2d)

In [ ]:
normalized_eucledianhuman36m.shape

In [ ]:
output_path='../inference_result_npy/'+base+'_2D_Proscrutes_Normalized.json'
save_to_json_2D(normalized_eucledianhuman36m, path, output_path)

# Inference Code (Convert only 1 Skeleton)

## 2D to 3D

In [30]:
def inferencealphaposeto3D_one(alpha2d,input_type="json"):
    converted=[]
    human2d=alpha2d.copy()
    human3d=alpha2d.copy()
    
    if (input_type=="json"):
        convert=np.asarray(data_converter(alpha2d))
    else:
        convert=alpha2d
        
    a=dim_to_use_2d.tolist()
    a.insert(18,28)
    a.insert(19,29)
    dim_to_use_2d_nose=np.asarray(a)

    human36m_output=map_alpha_to_human_classification(convert)
    human36m_output=human36m_output.astype('float')
    human2d['keypoints']=human36m_output[dim_to_use_2d_nose].tolist()

    human36m_alpha_example=map_alpha_to_human(convert)
    normalized=normalize_single_data(human36m_alpha_example,data_mean_2d,data_std_2d,dim_to_use_2d)
    normalized=normalized.astype('float')
    converted.append(normalized)
    converted=np.asarray(converted) 
    
    test_loader = DataLoader(
        dataset=Human36M_testing(converted,True),
        batch_size=1,
        shuffle=False,
        num_workers=0,
        pin_memory=True)

    #Doing Inference
    pred_result_all=test(test_loader, model, criterion, new_stat_3d) #All
    dim_use = np.hstack((np.arange(3), dim_to_use_3d))
    prediction=pred_result_all[0][0][dim_use]
    human3d['keypoints']=prediction.tolist()
        
    return human3d,human2d

In [31]:
path='../json_test/taoyuan_angle2.json'
f = open(path) 
data = json.load(f) 
# input_to=np.asarray(data_converter(data[0]))


In [32]:
data[0]

{'image_id': '00001.png',
 'category_id': 1,
 'pose_class': 'FallingDown_02',
 'keypoints': [1498.716064453125,
  483.880615234375,
  0.9190043807029724,
  1518.553466796875,
  470.65570068359375,
  0.9054306149482727,
  1478.878662109375,
  470.65570068359375,
  0.8983249068260193,
  1551.61572265625,
  437.5933837890625,
  0.7453621029853821,
  1445.81640625,
  437.5933837890625,
  0.7723419070243835,
  1591.29052734375,
  457.4307861328125,
  0.6885972023010254,
  1425.97900390625,
  444.2058410644531,
  0.7940304279327393,
  1630.96533203125,
  543.3927612304688,
  0.5835424661636353,
  1392.9166259765625,
  497.1055603027344,
  0.7696180939674377,
  1637.5777587890625,
  622.7423095703125,
  0.41547971963882446,
  1366.466796875,
  556.6177368164062,
  0.8615410923957825,
  1558.228271484375,
  662.4171142578125,
  0.7468500137329102,
  1459.041259765625,
  655.8046264648438,
  0.7562357187271118,
  1551.61572265625,
  821.1162109375,
  0.4312818944454193,
  1459.041259765625,
  8

In [33]:
# How to use function
output_3d=inferencealphaposeto3D_one(data[0])

torch.Size([1, 48])


In [34]:
# Output is array of 3D Pose baseline with shape (51,)
output_3d

{'image_id': '00001.png',
 'category_id': 1,
 'pose_class': 'FallingDown_02',
 'keypoints': [0.0,
  0.0,
  0.0,
  -97.96893722780383,
  -4.742379032845212,
  4.1372239011315415,
  -34.01093909260862,
  268.70586968369196,
  -121.40576070155869,
  -71.72193290420589,
  709.8621679434074,
  -107.84169791261655,
  98.43087330682758,
  4.035405071062647,
  -5.780692450820728,
  110.67299712624182,
  311.1741643559309,
  -137.26799283277106,
  135.27674832847737,
  761.7178265860889,
  -133.5079092155104,
  -14.734430609172723,
  -223.53460645083175,
  -7.834231579569881,
  -42.40683964958995,
  -474.9257487717407,
  -49.64888095461019,
  -127.23909158056924,
  -433.45405912142064,
  -134.79284931342528,
  -92.76190020896219,
  -464.67770156383165,
  -130.72633033515206,
  144.83576515707614,
  -466.3122447167458,
  -88.18001370934789,
  257.1410066497345,
  -278.58187682480616,
  -123.41827637025659,
  232.4732732530838,
  -87.6819763537415,
  -116.06111288072323,
  -201.82564678721866,
  

## Transformation

In [35]:
trans_path='../inference_result_npy/taoyuan_angle2_3D_Original to taoyuan_angle1_3D_Original_transformationvalue.pickle'

with open(trans_path, 'rb') as fp:
    trans = pickle.load(fp)

In [36]:
def find_transformation_single(trans,skeleton_):
    
    T=trans['T']
    b=trans['b']
    c=trans['c']
    
    skeleton=np.asarray(skeleton_['keypoints'])
    skeleton = skeleton.reshape(-1, 3)
    skeleton = (b * skeleton.dot(T)) + c
    skeleton=skeleton.reshape(51,)
    skeleton_['keypoints']=skeleton.tolist()
    
    return  skeleton_

In [37]:
find_transformation_single(trans,output_3d)

{'image_id': '00001.png',
 'category_id': 1,
 'pose_class': 'FallingDown_02',
 'keypoints': [-7.276597748864106,
  39.916814254366635,
  17.265527389410295,
  41.74932154696195,
  23.234229694402895,
  -50.11880153291253,
  60.41747175415564,
  286.0866247665738,
  -11.72704140356365,
  17.531047491415194,
  641.4379424782436,
  -149.34989559055035,
  -55.34278341440974,
  56.4666520766832,
  85.92262113000967,
  -9.657892230386816,
  342.2523878639466,
  83.99572153256734,
  -79.62957592091999,
  715.1769567817498,
  -8.841930700062541,
  32.81436428529889,
  -143.98212091185522,
  64.47917290411013,
  106.1186379330458,
  -344.22946718418456,
  125.90239479458364,
  203.29416873913337,
  -299.90716222209704,
  99.30511146742793,
  186.2736019792124,
  -322.43439153076673,
  128.2299230395787,
  33.26494076363154,
  -305.3811402794964,
  269.51883793413197,
  -24.299457606069662,
  -128.89831352236476,
  317.83113031746456,
  -39.498988004707414,
  23.510617866964804,
  251.8931611211

# Don't run (just experiment and random code) !

In [ ]:
unique_class=set([])
for a in new:
    unique_class.add(a['pose_class'])
    

In [ ]:
unique_class

In [ ]:
import json
f = open("../../../Falling Dataset/Thesis Experiment/data/taoyuan.json") 

# returns JSON object as  
# a dictionary 
data = json.load(f) 

In [ ]:
taoyuan_angle1=[]
taoyuan_angle2=[]

for a in data:
    if ((a['pose_class']=="FallingDown_01") or (a['pose_class']=="Standing_01")):
        taoyuan_angle1.append(a)
    else:
        taoyuan_angle2.append(a)


In [ ]:
with open("../../../Falling Dataset/Thesis Experiment/data/taoyuan_angle1.json", 'w') as fp:
    fp.write(json.dumps(taoyuan_angle1))

In [ ]:
with open("../../../Falling Dataset/Thesis Experiment/data/taoyuan_angle2.json", 'w') as fp:
    fp.write(json.dumps(taoyuan_angle2))